In [1]:
import numpy as np
import pandas as pd
from functools import reduce

In [54]:
#bakery = pd.read_csv("5000-out2.csv", names=np.arange(50))
bakery = pd.read_csv("75000-out2.csv", names=np.arange(50))
bakery

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74996,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
74997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
74998,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
74999,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [68]:
goods = pd.read_csv("goods.csv")

,Id,Flavor,Food,Price,Type
0,0,'Chocolate','Cake',8.95,'Food'
1,1,'Lemon','Cake',8.95,'Food'
2,2,'Casino','Cake',15.95,'Food'
3,3,'Opera','Cake',15.95,'Food'
4,4,'Strawberry','Cake',11.95,'Food'
5,5,'Truffle','Cake',15.95,'Food'
6,6,'Chocolate','Eclair',3.25,'Food'
7,7,'Coffee','Eclair',3.50,'Food'
8,8,'Vanilla','Eclair',3.25,'Food'
9,9,'Napoleon','Cake',13.49,'Food'


In [55]:
sum(bakery[0] & bakery[1] & bakery[2] & bakery[3])

4

In [56]:
#sum(bakery[0])
support = np.array([sum(bakery[i]) for i in bakery.columns])
rsupport = support/len(bakery[0])

In [57]:
support

array([6265, 6271, 5626, 6157, 6948, 6168, 3178, 8193, 3189, 6206, 3153,
       5784, 5117, 3177, 6362, 5690, 6221, 5529, 6987, 5685, 3253, 3243,
       6783, 5073, 5101, 3145, 3179, 6733, 7556, 5089, 3205, 5047, 6299,
       6166, 3243, 6943, 5077, 5790, 3183, 3307, 5118, 5081, 6871, 4685,
       5640, 7700, 6236, 5810, 4649, 5098])

In [58]:
rsupport

array([0.08353333, 0.08361333, 0.07501333, 0.08209333, 0.09264   ,
       0.08224   , 0.04237333, 0.10924   , 0.04252   , 0.08274667,
       0.04204   , 0.07712   , 0.06822667, 0.04236   , 0.08482667,
       0.07586667, 0.08294667, 0.07372   , 0.09316   , 0.0758    ,
       0.04337333, 0.04324   , 0.09044   , 0.06764   , 0.06801333,
       0.04193333, 0.04238667, 0.08977333, 0.10074667, 0.06785333,
       0.04273333, 0.06729333, 0.08398667, 0.08221333, 0.04324   ,
       0.09257333, 0.06769333, 0.0772    , 0.04244   , 0.04409333,
       0.06824   , 0.06774667, 0.09161333, 0.06246667, 0.0752    ,
       0.10266667, 0.08314667, 0.07746667, 0.06198667, 0.06797333])

In [59]:
def supports_vectorized(itemsets, matrix, minSup=0):
    supports_dict = {iset: 0 for iset in itemsets}
    for iset in itemsets:
        supports_dict[iset] = sum(reduce((lambda x,y: x&y), [matrix[item] for item in iset]))
    rm_list = []
    for iset in supports_dict.keys():
        if supports_dict[iset] < minSup:
            rm_list.append(iset)
    for iset in rm_list:
        supports_dict.pop(iset)
    return supports_dict
    #occurence_col = matrix[itemset[0]]
    #for i in range(1, len(itemset)):
    #    occurence_col = occurence_col & matrix[itemset[i]]
    #return sum(occurence_col)

In [60]:
def supports(itemsets, matrix, minSup=0):
    supports_dict = {iset: 0 for iset in itemsets}
    for idx in matrix.index:
        for iset in itemsets:
            row = matrix.loc[idx]
            if reduce((lambda x,y: x&y), [row[item] for item in iset]) == 1:
                supports_dict[iset] += 1
    rm_list = []
    for iset in supports_dict.keys():
        if supports_dict[iset] < minSup:
            rm_list.append(iset)
    for iset in rm_list:
        supports_dict.pop(iset)
    return supports_dict

In [62]:
def powerset_one_less(iterable):
    sets = []
    s = list(iterable)
    for i in range(len(iterable)):
        sets.append(frozenset(s[0:i]+s[i+1:]))
    return sets

In [63]:
def candidate_gen(set_list):
    C = []
    for f1 in set_list:
        for f2 in set_list:
            if len(f1.union(f2)) == len(f1) + 1:
                c = f1.union(f2)
                if c not in C:
                    flag = True
                    for subset in powerset_one_less(c):
                        if subset != f1 and subset != f2 and subset not in set_list:
                            flag = False
                    if flag:
                        C.append(c)
    
    return C

In [64]:
def apriori(matrix, total_itemset, minRelativeSup):
    all_supports_dict = {}
    skylines = {}
    minSup = minRelativeSup*matrix.shape[0]
    all_supports_dict[1] = supports_vectorized([frozenset([i]) for i in total_itemset], matrix, minSup)
    for singleton in all_supports_dict[1]:
        skylines[singleton] = 1
    k = 2
    while(k <= len(total_itemset) and len(all_supports_dict[k-1].values()) > 0):
        C_k = candidate_gen(all_supports_dict[k-1])
        all_supports_dict[k] = supports_vectorized(C_k, matrix, minSup)
        for ilist in all_supports_dict[k]:
            skylines[ilist] = 1
            for subset in powerset_one_less(ilist):
                skylines[subset] = 0
        k += 1
    skyline_list = []
    for itemset in skylines.keys():
        if skylines[itemset] == 1:
            skyline_list.append(itemset)
    return all_supports_dict, skyline_list

In [65]:
supports_dict,skyline_list = apriori(bakery, list(range(50)), .01)
print(skyline_list)

[frozenset({6}), frozenset({8}), frozenset({10}), frozenset({13}), frozenset({20}), frozenset({21}), frozenset({25}), frozenset({26}), frozenset({30}), frozenset({34}), frozenset({38}), frozenset({39}), frozenset({1, 19}), frozenset({9, 4}), frozenset({5, 22}), frozenset({44, 14}), frozenset({27, 28}), frozenset({33, 42}), frozenset({0, 2, 46}), frozenset({35, 18, 3}), frozenset({49, 15, 7}), frozenset({16, 32, 45}), frozenset({17, 29, 47}), frozenset({37, 7, 11, 45}), frozenset({48, 36, 12, 31}), frozenset({40, 41, 43, 23, 24})]


In [66]:
def association_rules(supports_dict, skyline_list, minConfidence = 0):
    confidence_dict = {}
    for iset in skyline_list:
        if (len(iset) >= 2):
            for item in iset:
                right_set = frozenset([item])
                left_set = iset.difference(right_set)
                total_support = supports_dict[len(iset)][iset]
                left_support = supports_dict[len(left_set)][left_set]
                confidence = total_support/left_support
                if confidence > minConfidence:
                    confidence_dict[(left_set, item)] = confidence
    return confidence_dict

In [74]:
association_rules_dict = association_rules(supports_dict, skyline_list, .5)

In [72]:
def goods_lookup(item_id, goods):
    row = goods.loc[item_id]
    return row["Flavor"][1:-1] + " " + row["Food"][1:-1]
goods_lookup(1, goods)

'Lemon Cake'

In [78]:
association_rules_formatted_list = []
for entry in association_rules_dict.keys():
    left_side = [goods_lookup(i, goods) for i in entry[0]]
    right_side = goods_lookup(entry[1], goods)
    confidence = association_rules_dict[entry]
    association_rules_formatted_list.append((left_side, right_side, confidence))
association_rules_formatted_list

[(['Napoleon Cake'], 'Strawberry Cake', 0.5214308733483726),
 (['Truffle Cake'], 'Gongolais Cookie', 0.5340466926070039),
 (['Bottled Water'], 'Berry Tart', 0.5026595744680851),
 (['Tuile Cookie'], 'Marzipan Cookie', 0.5054261514028586),
 (['Marzipan Cookie'], 'Tuile Cookie', 0.5672062973414526),
 (['Cheese Croissant'], 'Orange Juice', 0.5238404151800194),
 (['Casino Cake', 'Chocolate Coffee'], 'Chocolate Cake', 0.9474082482027999),
 (['Chocolate Cake', 'Chocolate Coffee'], 'Casino Cake', 0.7580986981531941),
 (['Chocolate Cake', 'Casino Cake'], 'Chocolate Coffee', 0.9395872420262664),
 (['Cherry Tart', 'Opera Cake'], 'Apricot Danish', 0.9477405471872118),
 (['Opera Cake', 'Apricot Danish'], 'Cherry Tart', 0.9553765106910443),
 (['Cherry Tart', 'Apricot Danish'], 'Opera Cake', 0.7742340532395781),
 (['Coffee Eclair', 'Blackberry Tart'], 'Single Espresso', 0.74697912852435),
 (['Single Espresso', 'Coffee Eclair'], 'Blackberry Tart', 0.9222423146473779),
 (['Single Espresso', 'Blackberry

In [ ]:
def mine_bakery_rules(filename, minRelativeSup, minConf, goods_filename):
    goods = pd.read_csv(goods_filename)
    bakery = pd.read_csv(filename, names=goods.index)
    supports_dict,skyline_list = apriori(bakery, list(goods.index), minRelativeSup)
    association_rules_dict = association_rules(supports_dict, skyline_list, .5)
    skylines_formatted = []
    for entry in skyline_list:
        skylines_formatted.append([goods_lookup(i, goods)])
    arules_formatted = []
    for entry in association_rules_dict.keys():
        left_side = [goods_lookup(i, goods) for i in entry[0]]
        right_side = goods_lookup(entry[1], goods)
        confidence = association_rules_dict[entry]
        arules_formatted.append((left_side, right_side, confidence))
    arules_formatted

In [80]:
skyline_list

[frozenset({6}),
 frozenset({8}),
 frozenset({10}),
 frozenset({13}),
 frozenset({20}),
 frozenset({21}),
 frozenset({25}),
 frozenset({26}),
 frozenset({30}),
 frozenset({34}),
 frozenset({38}),
 frozenset({39}),
 frozenset({1, 19}),
 frozenset({4, 9}),
 frozenset({5, 22}),
 frozenset({14, 44}),
 frozenset({27, 28}),
 frozenset({33, 42}),
 frozenset({0, 2, 46}),
 frozenset({3, 18, 35}),
 frozenset({7, 15, 49}),
 frozenset({16, 32, 45}),
 frozenset({17, 29, 47}),
 frozenset({7, 11, 37, 45}),
 frozenset({12, 31, 36, 48}),
 frozenset({23, 24, 40, 41, 43})]

In [ ]:
powerset_one_less((40,41,43,23,24))

In [22]:
frozenset([1]).union(frozenset([2]))

frozenset({1, 2})

In [34]:
bakery.shape

(5000, 50)